In [58]:
# Загружаем библиотеки
import pandas as pd # Датафреймы pandas
from surprise import SVD # Алгоритм сингулярного разложения
from surprise import Dataset, Reader # Структуры данных для surprise
from surprise import accuracy # Оценка точности моделей
from surprise.model_selection.split import KFold # Кросс-валидация

#### Загрузка данных (MovieLens 1M)

In [21]:
movies = pd.read_table('movies.dat', 
                       sep='::', 
                       header = None, 
                       names = ['MovieID', 'Title', 'Genres'], 
                       engine = 'python', 
                       encoding =  'ANSI')
ratings = pd.read_table('ratings.dat', 
                       sep='::', 
                       header = None, 
                       names = ['UserID', 'MovieID', 'Rating', 'Timestamp'], 
                       engine = 'python', 
                       encoding =  'ANSI')
users = pd.read_table('users.dat', 
                       sep='::', 
                       header = None, 
                       names = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], 
                       engine = 'python', 
                       encoding =  'ANSI')

#### Обработка данных

In [36]:
# Объединяем таблицы с названиями фильмов и рейтингами
movies_with_ratings = movies.join(ratings.set_index('MovieID'), on = 'MovieID').reset_index(drop = True)
# Отбрасываем строки с пропусками
movies_with_ratings.dropna(inplace = True)

In [41]:
# Формируем датасет для surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings['UserID'],
    'iid': movies_with_ratings['Title'],
    'rating': movies_with_ratings['Rating']
})

In [44]:
# Диапазон значений рейтинга
reader = Reader(rating_scale = (ratings['Rating'].min(), ratings['Rating'].max()))

In [45]:
# Рабочий датасет в формате surprise
data = Dataset.load_from_df(dataset, reader)

#### Модель SVD

In [65]:
# Итератор для разбивки данных на 5 частей
data_folds = KFold(n_splits = 5, random_state = 42)

In [140]:
# Вызываем алгоритм SVD
algo = SVD(n_epochs = 75, n_factors = 40, lr_all = 0.002, reg_all = 0.03, random_state = 42)

In [142]:
# Перебираем фолды данных
for trainset, testset in data_folds.split(data):
    # Обучаем модель
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Выводим показатель точности
    accuracy.rmse(predictions, verbose = True)

RMSE: 0.8641
RMSE: 0.8619
RMSE: 0.8626
RMSE: 0.8616
RMSE: 0.8617


In [143]:
# Полученные результаты удовлетворяют требованию RMSE <= 0.87